In [27]:
import numpy as np
import pandas as pd
import os
import sys
import shutil
import matplotlib.pyplot as plt
import pylab as pl
from keras import backend as K
from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History 
#from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import keras
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.layers.convolutional import Conv2D
from keras.layers import Dense
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [8 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\bijin\AppData\Local\Temp\pip-install-79cll1mn\sklearn_b99460c070bd451cb4117bc3b3728395\setup.py", line 10, in <module>
      LONG_DESCRIPTION = f.read()
    File "C:\Users\bijin\anaconda3\envs\tensorflow\lib\encodings\cp1252.py", line 23, in decode
      return codecs.charmap_decode(input,self.errors,decoding_table)[0]
  UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 7: character maps to <undefined>
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#! pip install -q kaggle

In [2]:
"""from google.colab import files
files.upload()"""

'from google.colab import files\nfiles.upload()'

In [3]:
"""!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list"""

'!rm -r ~/.kaggle\n!mkdir ~/.kaggle\n!mv ./kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n!kaggle datasets list'

In [4]:
#!kaggle datasets download -d hgunraj/covidxct -p /content/drive/MyDrive/dataset --unzip --force

In [8]:
# Total number of training images
import os
num_of_train_samples = 0
train=r'C:\Users\bijin\Downloads\archive\dataset\train'
val=r'C:\Users\bijin\Downloads\archive\dataset\val'
for train_dataset in os.listdir(train):
    in_folder = train + "/" + train_dataset 
    in_folder_list = os.listdir(in_folder)
    num_of_train_samples = num_of_train_samples + len(in_folder_list)
print("Number of training samples ",num_of_train_samples)

# Total number of validation images
num_of_validation_samples = 0
for validation_dataset in os.listdir(val):
    in_folder_val = val + "/" + validation_dataset
    in_folder_val_list = os.listdir(in_folder_val)
    num_of_validation_samples = num_of_validation_samples + len(in_folder_val_list)
print("Number of testing samples", num_of_validation_samples)

Number of training samples  61782
Number of testing samples 21036


In [13]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
batch_size = 16
target_names = ['COVID', 'PNEUMONIA', 'NORMAL']

# Defining image width and height respectively
img_rows = 224
img_cols = 224

In [15]:
train_generator = train_datagen.flow_from_directory(train,
                                                    target_size = (img_rows, img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(val,
                                                        target_size = (img_rows, img_cols),
                                                        batch_size = batch_size,
                                                        shuffle = False, class_mode='categorical')

Found 61782 images belonging to 3 classes.
Found 21036 images belonging to 3 classes.


In [16]:
"""def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    #This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("")
    else:
        print('Matriz de Confusão Não Normalizada')
    
    import itertools
    print(cm)
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Classe Real')
    plt.xlabel('Classe Predita')
    plt.tight_layout()"""

IndentationError: unexpected indent (2018445509.py, line 6)

In [17]:
steps_per_epoch = num_of_train_samples // batch_size
batch_size = 16
epoch = 15
learning_rate = 1e-5

In [28]:
from keras.applications import ResNet50
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, 
                   pooling='max', 
                   weights='imagenet'))
resnet_model.add(BatchNormalization())
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation = "relu"))
resnet_model.add(Dense(3, activation='softmax'))


# Summary: to find the number of parameters
resnet_model.layers[0].trainable=False
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 24,121,219
Trainable params: 529,411
Non-trainable params: 23,591,808
____________________________________

In [30]:
resnet_model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(learning_rate=1e-5),
              metrics=["accuracy"])

In [ ]:
resnet_training = resnet_model.fit(train_generator,
                               steps_per_epoch = steps_per_epoch,
                               epochs = epoch,
                               validation_data = validation_generator,
                               validation_steps = num_of_validation_samples // batch_size)

Epoch 1/15
3861/3861 [==============================] - 5732s 1s/step - loss: 0.8101 - accuracy: 0.6268 - val_loss: 0.7412 - val_accuracy: 0.6824
Epoch 2/15
3861/3861 [==============================] - 5470s 1s/step - loss: 0.6640 - accuracy: 0.7171 - val_loss: 0.6741 - val_accuracy: 0.7176
Epoch 3/15
3861/3861 [==============================] - 5122s 1s/step - loss: 0.6177 - accuracy: 0.7389 - val_loss: 0.5974 - val_accuracy: 0.7525
Epoch 4/15
2084/3861 [===============>..............] - ETA: 2:03:15 - loss: 0.5941 - accuracy: 0.7506

In [ ]:
training_accuracy_resnet      = resnet_training.history['accuracy'][-1]
training_loss_resnet          = resnet_training.history['loss'][-1]
validation_accuracy_resnet    = resnet_training.history['val_accuracy'][-1]
validation_loss_resnet        = resnet_training.history['val_loss'][-1]
print("Training Accuracy ResNet   :", training_accuracy_resnet)
print("Training Loss ResNet       :", training_loss_resnet)
print("Validation Accuracy ResNet :", validation_accuracy_resnet)
print("Validation Loss ResNet     :", validation_loss_resnet)